In [1]:
import os
cwd =  os.getcwd().replace("notebooks/research","")
os.chdir(cwd)

In [2]:
# --- Robust notebook shim for legacy joblib artifacts expecting `encoders.*` ---
import sys, types, numpy as np

# Create/replace a lightweight 'encoders' module in sys.modules
enc_mod = types.ModuleType("encoders")

try:
    from sentence_transformers import SentenceTransformer
except Exception as e:
    SentenceTransformer = None
    print("NOTE: sentence-transformers not available:", e)

class _SBERTBase:
    """
    Compat shim implementing the sklearn Transformer API expected by saved Pipelines.
    Handles pickles that don't call __init__ and are missing attributes.
    Provides both class names: SBERTEncoder and SBERTFeaturizer.
    """
    # NOTE: __init__ might not be called during unpickle; use _ensure_attrs() everywhere.
    def __init__(self, model="sentence-transformers/all-MiniLM-L6-v2", **kwargs):
        self.model_name = model
        self._enc = None
        self._kwargs = kwargs

    def _ensure_attrs(self):
        # Add any attributes that might be missing from legacy pickles
        if not hasattr(self, "model_name") or self.model_name is None:
            self.model_name = "sentence-transformers/all-MiniLM-L6-v2"
        if not hasattr(self, "_enc"):
            self._enc = None
        if not hasattr(self, "_kwargs"):
            self._kwargs = {}

    def _ensure_encoder(self):
        self._ensure_attrs()
        if self._enc is None:
            if SentenceTransformer is None:
                raise RuntimeError(
                    "sentence-transformers not installed in this kernel; "
                    "pip install sentence-transformers && restart kernel"
                )
            self._enc = SentenceTransformer(self.model_name)

    # sklearn API
    def fit(self, X, y=None):
        self._ensure_attrs()
        return self

    def transform(self, X):
        self._ensure_encoder()
        return np.asarray(self._enc.encode(list(X), show_progress_bar=False))

    # some older code may call .encode directly; alias it
    def encode(self, X):
        return self.transform(X)

# Expose both legacy names on the encoders module
class SBERTEncoder(_SBERTBase): ...
class SBERTFeaturizer(_SBERTBase): ...

enc_mod.SBERTEncoder = SBERTEncoder
enc_mod.SBERTFeaturizer = SBERTFeaturizer
sys.modules["encoders"] = enc_mod

# Make sure your package code is importable too (if needed)
import pathlib
if str(pathlib.Path("src").resolve()) not in sys.path:
    sys.path.append(str(pathlib.Path("src").resolve()))
print("encoders shim ready (SBERTEncoder + SBERTFeaturizer) and sys.path configured")

encoders shim ready (SBERTEncoder + SBERTFeaturizer) and sys.path configured


In [25]:
import joblib
from pathlib import Path

def load_mapper():
    for name in [".artifacts/arc_mapper.joblib"]:
        p = Path(name).resolve()
        if p.exists():
            print("Loading:", p.as_posix())
            return joblib.load(p.as_posix())
    raise FileNotFoundError("No mapper artifact found in .artifacts/")

mapper = load_mapper()
print(mapper)


Loading: /Users/ian_moore/repos/micro-lm/.artifacts/arc_mapper.joblib
Pipeline(steps=[('sbertencoder', <__main__.SBERTEncoder object at 0x36382b040>),
                ('calibratedclassifiercv',
                 CalibratedClassifierCV(cv=3,
                                        estimator=LogisticRegression(C=8.0,
                                                                     class_weight='balanced',
                                                                     max_iter=2000,
                                                                     random_state=0),
                                        method='isotonic'))])


In [179]:
prompt = "flip the grid horizontally"
prompt = "Rotate the grid 90 degrees clockwise, then flip it horizontally"
pred  = mapper.predict([prompt])[0]
probs = mapper.predict_proba([prompt])[0]
print("Predicted:", pred)
print("Top-3:", sorted(zip(mapper.classes_, probs), key=lambda t: t[1], reverse=True)[:3])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Predicted: rotate
Top-3: [('rotate', 1.0), ('flip_h', 0.0), ('flip_v', 0.0)]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [79]:
def mapper_fn(prompt, labels=("flip_h","flip_v","rot90"), mapper=mapper):
    probs = mapper.predict_proba([prompt])[0]
    res = sorted(zip(mapper.classes_, probs), key=lambda t: t[1], reverse=True)
    return {res[0][0]:res[0][1], res[1][0]:res[1][1], res[2][0]:res[2][1]}

# def mapper_fn(prompt, labels=("flip_h","flip_v","rot90"), temperature=0.0):
#     return {'flip_h': 0.9375, 'flip_v': 0.06250000000000001, 'rot90': 0.0}

In [164]:
import numpy as np
from scipy.ndimage import gaussian_filter1d

PROTO_WIDTH = 160  # keep consistent with WDD_CFG["proto_width"]

def _upsample(x, L):
    x = np.asarray(x, dtype=np.float32)
    if len(x) == L: return x
    xp = np.linspace(0, len(x)-1, num=len(x))
    xq = np.linspace(0, len(x)-1, num=L)
    return np.interp(xq, xp, x)

def _smooth_norm(x, sigma=3.5):
    x = gaussian_filter1d(x.astype(np.float32), sigma=sigma, mode="nearest")
    x -= x.min()
    x /= (x.max() - x.min() + 1e-8)
    return x

def _scan_cols(A, B):
    # per-column Hamming match
    return np.array([np.mean(A[:, j] == B[:, j]) for j in range(A.shape[1])], np.float32)

def _scan_rows(A, B):
    # per-row Hamming match
    return np.array([np.mean(A[i, :] == B[i, :]) for i in range(A.shape[0])], np.float32)

def _rot90_proxy(G):
    # diagonal-band match against 90° rotation (CCW)
    R = np.rot90(G, 1)
    H, W = G.shape
    vals = []
    for k in range(-(H-1), W):
        d1 = np.diagonal(G, offset=k)
        d2 = np.diagonal(R, offset=k)
        m = min(len(d1), len(d2))
        if m > 0:
            vals.append(np.mean(d1[:m] == d2[:m]))
    return np.array(vals if vals else [0.0], np.float32)

def traces_from_grid(grid, L=PROTO_WIDTH):
    G = np.asarray(grid, dtype=int)
    Gh = G[:, ::-1]      # flip_h hypothesis (left↔right)
    Gv = G[::-1, :]      # flip_v hypothesis (top↕bottom)

    # axis-specific matches
    hc = _scan_cols(G, Gh); hr = _scan_rows(G, Gh)  # matches under flip_h
    vc = _scan_cols(G, Gv); vr = _scan_rows(G, Gv)  # matches under flip_v

    # CONTRASTIVE traces:
    #  - flip_h should be strong when column-wise evidence for h-flip is high
    #    AND row-wise evidence for v-flip is low → hc - vr
    #  - flip_v should be strong when row-wise evidence for v-flip is high
    #    AND col-wise evidence for h-flip is low → vr - hc
    raw_h = hc - vr
    raw_v = vr - hc

    # rot90 proxy
    raw_r = _rot90_proxy(G)

    # upsample → smooth → normalize
    th = _smooth_norm(_upsample(raw_h, L))
    tv = _smooth_norm(_upsample(raw_v, L))
    tr = _smooth_norm(_upsample(raw_r, L))

    # tiny ε patterns to avoid pathological flat ties on micro-grids
    eps = 1e-3
    lin = np.linspace(0, 1, L, dtype=np.float32)
    th = (1-eps)*th + eps*lin
    tv = (1-eps)*tv + eps*(1-lin)
    tr = (1-eps)*tr + eps*(0.5 + 0.5*np.sin(2*np.pi*lin))

    return {"flip_h": th, "flip_v": tv, "rot90": tr}


In [93]:
from typing import Dict, Optional, Tuple, List

PRIMS = ["flip_h","flip_v","rotate"]

def gaussian_bump(T, center, width, amp=1.0):
    t = np.arange(T)
    sig2 = (width/2.355)**2  # FWHM→σ
    return amp * np.exp(-(t-center)**2 / (2*sig2))

# ----------------------------
# Synthetic ARC-like generator
# ----------------------------
def make_synthetic_traces(rng, T=720, noise=0.02, cm_amp=0.02, overlap=0.5,
                          amp_jitter=0.4, distractor_prob=0.4,
                          tasks_k: Tuple[int,int]=(1,3)) -> Tuple[Dict[str,np.ndarray], List[str]]:
    k = int(rng.integers(tasks_k[0], tasks_k[1]+1))
    tasks = list(rng.choice(PRIMS, size=k, replace=False))
    rng.shuffle(tasks)
    base = np.array([0.20, 0.50, 0.80]) * T
    centers = ((1.0 - overlap) * base + overlap * (T * 0.50)).astype(int)
    width = int(max(12, T * 0.10))
    t = np.arange(T)
    cm = cm_amp * (1.0 + 0.2 * np.sin(2*np.pi * t / max(30, T//6)))

    traces = {p: np.zeros(T, float) for p in PRIMS}
    for i, prim in enumerate(tasks):
        c = centers[i % len(centers)]
        amp = max(0.25, 1.0 + rng.normal(0, amp_jitter))
        c_jit = int(np.clip(c + rng.integers(-width//5, width//5 + 1), 0, T-1))
        traces[prim] += gaussian_bump(T, c_jit, width, amp=amp)

    for p in PRIMS:
        if p not in tasks and rng.random() < distractor_prob:
            c = int(rng.uniform(T*0.15, T*0.85))
            amp = max(0.25, 1.0 + rng.normal(0, amp_jitter))
            traces[p] += gaussian_bump(T, c, width, amp=0.9*amp)

    for p in PRIMS:
        traces[p] = np.clip(traces[p] + cm, 0, None)
        traces[p] = traces[p] + rng.normal(0, noise, size=T)
        traces[p] = np.clip(traces[p], 0, None)

    return traces, tasks

In [159]:
def traces_from_synthetic(label: str, T: int = 160):
    """Deterministic single-task traces (no noise, no distractors) for a given primitive label."""
    rng = np.random.default_rng(0)
    # 1 task, zero noise/common-mode, no distractors
    traces, tasks = make_synthetic_traces(
        rng, T=T, noise=0.0, cm_amp=0.0, overlap=0.5,
        amp_jitter=0.0, distractor_prob=0.0, tasks_k=(1,1)
    )
    # Force the task to the requested label by moving the bump to that channel
    if label in PRIMS:
        # clear all channels then copy the nonzero bump into target channel
        # find the single active channel the generator used
        active = [p for p,v in traces.items() if np.max(v) > 0.0]
        src = active[0] if active else PRIMS[0]
        bump = traces[src].copy()
        for p in PRIMS: traces[p] = np.zeros_like(bump)
        traces[label] = bump
    return traces

In [178]:
#traces_from_grid(grid)

In [158]:
traces2, true_tasks2 = make_synthetic_traces(
    rng, T=160, noise=0.0, cm_amp=0.0, overlap=0.0, amp_jitter=0.0, distractor_prob=0.0,
    tasks_k=(3,3)
)
print("TRUE:", true_tasks2)   # e.g. ['flip_v','rotate'] in time order
# print("REPORT:", *geodesic_parse_report(traces2, sigma=9, proto_width=160))

TRUE: ['flip_v', 'flip_h', 'rotate']


In [101]:
# 1) make deterministic, noise-free synthetic traces (single task)
rng = np.random.default_rng(123)
traces, true_tasks = make_synthetic_traces(
    rng,
    T=160,               # match your proto_width for convenience
    noise=0.0,           # <-- zero noise
    cm_amp=0.0,          # <-- kill common-mode drift
    overlap=0.0,         # keep bumps separated if you choose >1 task
    amp_jitter=0.0,      # remove amplitude jitter
    distractor_prob=0.0, # no distractor bumps
    tasks_k=(1,1)        # exactly one primitive
)

In [166]:
grid

array([[3, 0, 3, 6],
       [3, 0, 3, 6],
       [3, 2, 1, 3],
       [3, 2, 0, 5]])

In [177]:
import numpy as np
# from wdd_arc_impl_patched import load_priors_npz, run_arc_wdd
from notebooks.research.wdd_arc_impl_patched import  load_priors_npz, run_arc_wdd

prompt = "flip the grid horizontally"
grid = np.array([[3,0,3,],[3,2,1],[3,2,0]], dtype=int)

# run without priors first
res = run_arc_wdd(prompt, grid, mapper_fn, traces_from_grid, priors, use_priors=True)
print(res["verdict"], res.get("label"), res.get("keep"), res.get("order"))
res

NameError: name 'priors' is not defined

In [80]:
mapper_fn("flip the grid vertcially")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'flip_v': 0.5515621633268692, 'flip_h': 0.4484378366731308, 'rotate': 0.0}

In [54]:
res = run_arc_wdd(prompt, grid, mapper_fn, traces_from_grid,
                  priors=None, use_priors=False)
print(res)

{'verdict': 'PASS', 'label': 'flip_h', 'keep': ['flip_h'], 'order': ['flip_h'], 'out': array([[1, 0, 3],
       [1, 2, 3],
       [0, 2, 3]]), 'mapper': {'flip_h': 0.9375, 'flip_v': 0.06250000000000001, 'rot90': 0.0}, 'route': ['flip_h', 'flip_v']}


In [40]:
mp = mapper_fn(prompt)
print("mapper probs:", mp, "top1:", max(mp, key=mp.get))

tr = traces_from_grid(grid)
print("trace keys:", list(tr))
for k,v in tr.items():
    print(k, "len", len(v), "nan?", np.isnan(v).any(), "std", np.std(v), "min/max", (float(np.min(v)), float(np.max(v))))


mapper probs: {'flip_h': 0.9375, 'flip_v': 0.06250000000000001, 'rot90': 0.0} top1: flip_h
trace keys: ['flip_h', 'flip_v', 'rot90']
flip_h len 160 nan? False std 0.21490313 min/max (0.0010000000474974513, 0.9991006255149841)
flip_v len 160 nan? False std 0.22710231 min/max (0.0, 1.0)
rot90 len 160 nan? False std 0.25631213 min/max (0.4998902380466461, 1.4987125396728516)


In [43]:
import numpy as np
A = np.arange(12).reshape(3,4)
flip_h_exec = A[:, ::-1]
flip_v_exec = A[::-1, :]
print("A:\n", A)
print("flip_h (L↔R):\n", flip_h_exec)
print("flip_v (T↕B):\n", flip_v_exec)

A:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
flip_h (L↔R):
 [[ 3  2  1  0]
 [ 7  6  5  4]
 [11 10  9  8]]
flip_v (T↕B):
 [[ 8  9 10 11]
 [ 4  5  6  7]
 [ 0  1  2  3]]


In [44]:
import numpy as np

A = np.arange(12).reshape(3,4)
assert np.allclose(A[:, ::-1], np.array([[3,2,1,0],[7,6,5,4],[11,10,9,8]]))
assert np.allclose(A[::-1, :], np.array([[8,9,10,11],[4,5,6,7],[0,1,2,3]]))

# mapper sanity:
mp = mapper_fn("flip the grid horizontally")
assert "flip_h" in mp and "flip_v" in mp, "mapper must expose both"
# if your policy is 'horizontal' == left-right:
assert max(mp, key=mp.get) == "flip_h", "mapper text->label semantics mismatch"
